<a href="https://colab.research.google.com/github/tomonari-masada/course2021-stats1/blob/main/gradient_based_training_of_PLSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 勾配法によるPLSIの学習

* EMアルゴリズムではなく、勾配法でPLSIの学習を行ってみる
 * 偏微分係数をゼロとおいた方程式を解く、という方法ではなく、勾配を使って、パラメータを更新する

* 例題として、ライブドア・ニュース・コーパスのトピック分析を行う

## MeCabのインストール
* 今回、日本語データを使うので、この作業が必要になっている

In [ ]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
#!pip install fugashi ipadic

In [ ]:
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

## データの取得

* ライブドア・ニュース・コーパスの前処理については下の記事を参考にした。
 * https://tech.fusic.co.jp/posts/2021-04-23-bert-multi-classification/

In [ ]:
import os
import urllib.request
import re
import csv
import tarfile
import numpy as np
import pandas as pd

# データのダウンロード（カレントディレクトリに圧縮ファイルがダウンロードされる）
urllib.request.urlretrieve("https://www.rondhuit.com/download/ldcc-20140209.tar.gz", "ldcc-20140209.tar.gz")

# ダウンロードした圧縮ファイルのパスを設定
tgz_fname = "ldcc-20140209.tar.gz" 

#処理をした結果を保存するファイル名 
tsv_fname = "all_text.tsv" 

In [ ]:
fname_class_list = {
  "dokujo-tsushin": [],
  "it-life-hack": [],
  "kaden-channel": [],
  "livedoor-homme": [],
  "movie-enter": [],
  "peachy": [],
  "smax": [],
  "sports-watch": [],
  "topic-news": []
}
target_genres = list(fname_class_list.keys())

* 記号などを除きつつ、ニュース記事の本文を取得する

In [ ]:
def remove_brackets(inp):
  brackets_tail = re.compile('【[^】]*】$')
  brackets_head = re.compile('^【[^】]*】')
  output = re.sub(brackets_head, '', re.sub(brackets_tail, '', inp))
  return output

def read_body(f):
  # 2行スキップ
  next(f) # URL
  next(f) # タイムスタンプ
  next(f) # タイトル
  lines = [line.decode('utf-8').strip() for line in f]
  body = ' '.join(lines)
  body = remove_brackets(body)
  return body

# all_text.tsvを作る
with tarfile.open(tgz_fname) as tf:
  # 対象ファイルの選定
  for ti in tf:
    """
    ・ライセンスファイルはスキップ
    ・genre内のtxt意外ならスキップ
    ・txtファイル意外ならスキップ
    ・用意したgenre意外ならスキップ
    """
    if "LICENSE.txt" in ti.name:
      continue
    if len(ti.name.split('/')) < 3:
      continue
    if not ti.name.endswith(".txt"):
      continue
    genre = ti.name.split('/')[1]
    if not genre in target_genres:
      continue
        
    genre_index = target_genres.index(genre)
    fname_class_list[target_genres[genre_index]].append(ti.name)

  with open(tsv_fname, "w") as wf:
    writer = csv.writer(wf, delimiter='\t')
    for i, genre in enumerate(target_genres):
      for fname in fname_class_list[genre]:
        f = tf.extractfile(fname)
        row = [genre, i, read_body(f)]
        writer.writerow(row)

In [ ]:
# 作成したデータの読み込み
df = pd.read_csv("all_text.tsv", delimiter='\t', header=None, names=['media_name', 'label', 'body'])
df = df.dropna(how='any') # nanのところは落とす

# データの確認
print(f'データサイズ： {df.shape}')
display(df.sample(10))

## トークン化
* MeCabを使って形態素解析することで、テキストをトークン化する

In [ ]:
import MeCab

m = MeCab.Tagger()

In [ ]:
m.parse(df.body[0])

In [ ]:
def tokenize(text):
  tokens = []
  for line in m.parse(text).splitlines():
    fields = line.split()
    if len(fields) != 2: continue
    subfields = fields[1].split(',')
    if len(subfields) != 9: continue
    if subfields[0] in ['記号', '助詞', '助動詞', '連体詞', '副詞']: continue
    token = subfields[6]
    if token == '*':
      token = fields[0]
    tokens.append(token)
  return ' '.join(tokens)

In [ ]:
corpus = []
for body in df.body:
  corpus.append(tokenize(body))

* 文書数は7,367件

In [ ]:
len(corpus)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=20, max_df=0.2)
X = vectorizer.fit_transform(corpus)
vocab = vectorizer.get_feature_names_out()

In [ ]:
len(vocab)

## PLSIの学習
* $\log (\mathcal{D} ) = \sum_{d=1}^D \log p(\mathbf{x}_d) = \sum_{d=1}^D \sum_{i=1}^{N_d} \log \bigg( \sum_{k=1}^K \phi_{k,x_{d,i}} \theta_{d,k} \bigg)$を直接最大化する
* 尤度の式の$\sum_{k=1}^K \phi_{k,x_{d,i}} \theta_{d,k}$の部分は、文書$d$の$i$番目に単語$x_{d,i}$が出現する確率を表す。この確率は、
 * 各トピック$k$におけるその単語の出現確率$\phi_{k,x_{d,i}}$を、
 * 文書$d$における各トピックの混合率$\theta_{d,k}$で重み付けして加算することで、求められている。
* 以下の二つの行列を準備する
 * $\boldsymbol{\Theta}$: 第$d$行、第$k$列が、文書$d$におけるトピック$k$の混合率を表す。
 * $\boldsymbol{\Phi}$: 第$k$行、第$w$列が、トピック$k$における単語$w$の出現確率を表す。
* すると、PLSIは、文書$d$に単語$w$が出現する確率として、$\boldsymbol{\Theta}\boldsymbol{\Phi}$の第$d$行第$w$列の値を使うモデルだと、解釈できる。

* PyTorchを使って実装する

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

* 各ニュース記事における各単語の出現頻度をPyTorchのテンソルに変換

In [ ]:
word_freqs = torch.from_numpy(X.toarray()).type(torch.float32)

* PLSIのパラメータを、微分可能なテンソルとして準備

In [ ]:
n_components = 20
theta = torch.randn(len(corpus), n_components, requires_grad=True)
phi = torch.randn(n_components, len(vocab), requires_grad=True)

In [ ]:
optimizer = torch.optim.Adam([theta, phi], lr=1.0)

* ミニバッチ式ではなく、バッチ式で学習を進めている
 * ミニバッチにしなくてもメモリが足りるため。

In [ ]:
for epoch in range(100):
  normalized_theta = torch.nn.functional.softmax(theta, dim=1)
  normalized_phi = torch.nn.functional.softmax(phi, dim=1)
  word_probs = normalized_theta @ normalized_phi

  loss = - (word_freqs * word_probs.log()).sum()
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()

  print(f'epoch {epoch} | loss {loss}')

## トピック語の確認

* 各々の$k$について、$\phi_{k,w}$が大きい上位30単語を調べる

In [ ]:
n_topic_words = 30
for k in range(n_components):
  print(' '.join([vocab[idx] for idx in torch.argsort(phi[k], descending=True)[:n_topic_words]]))

# 課題
* このバッチ最適化によるパラメータ推定を、ミニバッチ最適化に書き換えてみよう。
 * 同じ方法でlossを計算して比較することで、よりlossを小さくできるか、確認してみる。
* もしくは、授業で説明したEMアルゴリズムによる推定を実装し、今回のバッチ最適化と性能比較してみよう。
* もちろん、使用するフレームワークはPyTorchでなくてもいいです。